In [33]:
import cv2
import numpy as np

step = 0
start_point = None
end_point = None
cropping = False

lowers =[]
uppers =[]

def crop_phase():
    # Load an image
    cap = cv2.VideoCapture("Camp.mp4")

    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the video has ended
    if not ret:
        print("Error: Could not read frame from video.")
        exit()

    frame = cv2.resize(frame, (500, 500))

    # Create a copy of the original frame
    clone = frame.copy()

    # Define the starting and ending points for cropping
   

    # Define the mouse callback function
    def mouse_crop(event, x, y, flags, param):
        global start_point, end_point, cropping
        
        # Start cropping when the left mouse button is pressed
        if event == cv2.EVENT_LBUTTONDOWN:
            start_point = (x, y)
            print("Start point: ", start_point)
            cropping = True
        
        # End cropping when the left mouse button is released
        elif event == cv2.EVENT_LBUTTONUP:
            end_point = (x, y)
            cropping = False
            print("End point: ", end_point)
            
            # Draw the rectangle around the cropped area
            cv2.rectangle(clone, start_point, end_point, (0, 255, 0), 2)
            cv2.imshow("Cropped Image", clone)
            
            # Crop the original frame using the start and end points
            cropped = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]
            cv2.imshow("Cropped Image", cropped)

    # Create a window and set the mouse callback function
    cv2.namedWindow("Cropped Image")
    cv2.setMouseCallback("Cropped Image", mouse_crop)

    # Display the original frame
    cv2.imshow("Cropped Image", clone)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        # Close all windows
        cv2.destroyAllWindows()

        filter_phase(0)

def filter_phase(iteration):

    # Load an image
    cap = cv2.VideoCapture("Camp.mp4")

    # Create a window to display the sliders
    cv2.namedWindow("HSV Mask",cv2.WINDOW_AUTOSIZE)

    # Create trackbars for hue, saturation, and value
    cv2.createTrackbar("LHue", "HSV Mask", 0, 360, lambda x: x)
    cv2.createTrackbar("UHue", "HSV Mask", 360, 360, lambda x: x)
    cv2.createTrackbar("Saturation", "HSV Mask", 0, 255, lambda x: x)
    cv2.createTrackbar("Value", "HSV Mask", 0, 255, lambda x: x)

    while True:
        
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the video has ended
        if not ret:
            cap = cv2.VideoCapture("Camp.mp4")
            continue

        frame = cv2.resize(frame, (500, 500))

        frame = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]


        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Get the current values of the trackbars
        uhue = cv2.getTrackbarPos("UHue", "HSV Mask")
        lhue = cv2.getTrackbarPos("LHue", "HSV Mask")
        sat = cv2.getTrackbarPos("Saturation", "HSV Mask")
        val = cv2.getTrackbarPos("Value", "HSV Mask")

        # Define the range of HSV values to mask
        lower = np.array([lhue, sat, val])
        upper = np.array([uhue, 255, 255])

        # Create a mask for the desired HSV range
        mask = cv2.inRange(hsv_frame, lower, upper)

        # Apply the mask to the original image
        result = cv2.bitwise_and(frame, frame, mask=mask)

        # Display the original image
        cv2.imshow("Original", frame)

        # Display the result of applying the mask
        cv2.imshow("HSV Mask", result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            lowers.append(lower)
            uppers.append(upper)
            break

    # Release the video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()
    if(iteration!=2):
        filter_phase(iteration+1)
    else:
        detection_phase()

def detection_phase():
    # Load an image
    cap = cv2.VideoCapture("Camp.mp4")

    cv2.namedWindow("HSV Mask")
    cv2.createTrackbar("Area", "HSV Mask", 0, 100, lambda x: x)
    cv2.createTrackbar("Area2", "HSV Mask", 0, 1000, lambda x: x)

    while True:
        
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the video has ended
        if not ret:
            cap = cv2.VideoCapture("Camp.mp4")
            continue

        frame = cv2.resize(frame, (500, 500))

        frame = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

        # Convert the image to the HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        masks = []
        for i in range(3):
            
            # Create a mask for the desired HSV range
            mask = cv2.inRange(hsv_frame, lowers[i], uppers[i])
            masks.append(mask)
        
        final_mask = masks[0]
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)
            
        result = cv2.bitwise_and(frame, frame, mask=final_mask)
        result2 = result.copy()
        area_threshold1 = cv2.getTrackbarPos("Area", "HSV Mask")
        area_threshold2 = cv2.getTrackbarPos("Area2", "HSV Mask")
        contours,_ = cv2.findContours(final_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        count = 0
        red_count = 0
        for c in contours:
            if area_threshold1 < cv2.contourArea(c) < area_threshold2:
                total_pixel = 0
                red_pixel = 0
                for point in c:
                    x, y = point[0]
                    # if frameHSV[y][x][0] > frameHSV[y][x][1] and frameHSV[y][x][0] > frameHSV[y][x][2]:
                    #      red_pixel +=1
                    total_pixel +=1
                if (red_pixel / total_pixel) > 0.5:
                    print("red dominant")
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(result2,(x-10,y-10), ((x+w)+10,(y+h)+10),(0,255,0),1)
                count+=1
                red_count+=1
        text = "Number of boxes: {}".format(count)
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        text_pos = (10, result2.shape[0] - 10)
        cv2.rectangle(result2, (text_pos[0], text_pos[1] - text_size[1]), (text_pos[0] + text_size[0], text_pos[1]), (0, 0, 0), -1)
        cv2.putText(result2, "Number of boxes: {} ({})".format(count,red_count), (10, result2.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display the result of applying the mask
        cv2.imshow("HSV Mask", result2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release the video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()

crop_phase()









Start point:  (31, 238)
End point:  (330, 353)


In [48]:
red_count = 0
count = 0
trig = 0
def detection_phase():
    global red_count, count, trig
    # Load an image
    cap = cv2.VideoCapture("Camp.mp4")

    cv2.namedWindow("HSV Mask")
    cv2.createTrackbar("Area", "HSV Mask", 0, 100, lambda x: x)
    cv2.createTrackbar("Area2", "HSV Mask", 0, 1000, lambda x: x)

    while True:
        
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the video has ended
        if not ret:
            cap = cv2.VideoCapture("Camp.mp4")
            continue

        frame = cv2.resize(frame, (500, 500))

        frame = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

        # Convert the image to the HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        masks = []
        for i in range(3):
            
            # Create a mask for the desired HSV range
            mask = cv2.inRange(hsv_frame, lowers[i], uppers[i])
            masks.append(mask)
        
        final_mask = masks[0]
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)
            
        result = cv2.bitwise_and(frame, frame, mask=final_mask)
        result2 = result.copy()
        area_threshold1 = cv2.getTrackbarPos("Area", "HSV Mask")
        area_threshold2 = cv2.getTrackbarPos("Area2", "HSV Mask")
        contours,_ = cv2.findContours(final_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        _count_ = 0
        _red_count_ = 0
        for c in contours:
            if area_threshold1 < cv2.contourArea(c) < area_threshold2:
                total_pixel = 0
                red_pixel = 0
                for point in c:
                    x, y = point[0]
                    if hsv_frame[y][x][0] > hsv_frame[y][x][1] and hsv_frame[y][x][0] > hsv_frame[y][x][2]:
                         red_pixel +=1
                    total_pixel +=1
                    
                if (red_pixel / total_pixel) > 0.5:
                    _red_count_+=1
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(result2,(x-10,y-10), ((x+w)+10,(y+h)+10),(0,255,0),1)
                _count_+=1
        if(red_count != _red_count_ or count != _count_ ):
            trig+=1
        else:
            trig = 0
        if(trig == 15):
            trig = 0
            red_count = _red_count_
            count = _count_
                
        text = "Number of boxes: {} ({})".format(count,red_count)
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        text_pos = (10, result2.shape[0] - 10)
        cv2.rectangle(result2, (text_pos[0], text_pos[1] - text_size[1]), (text_pos[0] + text_size[0], text_pos[1]), (0, 0, 0), -1)
        cv2.putText(result2, "Number of boxes: {} ({})".format(count,red_count), (10, result2.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display the result of applying the mask
        cv2.imshow("HSV Mask", result2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

detection_phase()

In [ ]:
ct = [False for i in range(3)]

red_count = 0
count = 0
trig = 0
def detection_phase():
    global red_count, count, trig, ct
    # Load an image
    cap = cv2.VideoCapture("Camp.mp4")

    cv2.namedWindow("HSV Mask")
    cv2.createTrackbar("Area", "HSV Mask", 0, 100, lambda x: x)
    cv2.createTrackbar("Area2", "HSV Mask", 0, 1000, lambda x: x)

    while True:
        
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the video has ended
        if not ret:
            cap = cv2.VideoCapture("Camp.mp4")
            continue

        frame = cv2.resize(frame, (500, 500))

        frame = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

        # Convert the image to the HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        masks = []
        for i in range(3):
            
            # Create a mask for the desired HSV range
            mask = cv2.inRange(hsv_frame, lowers[i], uppers[i])
            masks.append(mask)
        index = 1
        final_mask = masks[0]
        for mask in masks[1:]:
            if not ct[index]:
                final_mask = cv2.bitwise_or(final_mask, mask)
            index+=1
        result = cv2.bitwise_and(frame, frame, mask=final_mask)
        result2 = result.copy()
        area_threshold1 = cv2.getTrackbarPos("Area", "HSV Mask")
        area_threshold2 = cv2.getTrackbarPos("Area2", "HSV Mask")
        contours,_ = cv2.findContours(final_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        _count_ = 0
        _red_count_ = 0
        
        for c in contours:
            
            if area_threshold1 < cv2.contourArea(c) < area_threshold2:
                total_pixel = 0
                red_pixel = 0
                for point in c:
                    x, y = point[0]
                    if hsv_frame[y][x][0] > hsv_frame[y][x][1] and hsv_frame[y][x][0] > hsv_frame[y][x][2]:
                         red_pixel +=1
                    total_pixel +=1
                    
                if (red_pixel / total_pixel) > 0.5:
                    _red_count_+=1
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(result2,(x-10,y-10), ((x+w)+10,(y+h)+10),(0,255,0),1)
                _count_+=1
        if(red_count != _red_count_ or count != _count_ ):
            trig+=1
        else:
            trig = 0
        if(trig == 15):
            trig = 0
            red_count = _red_count_
            count = _count_
                
        text = "Number of boxes: {} ({})".format(count,red_count)
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        text_pos = (10, result2.shape[0] - 10)
        cv2.rectangle(result2, (text_pos[0], text_pos[1] - text_size[1]), (text_pos[0] + text_size[0], text_pos[1]), (0, 0, 0), -1)
        cv2.putText(result2, "Number of boxes: {} ({})".format(count,red_count), (10, result2.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display the result of applying the mask
        cv2.imshow("HSV Mask", result2)
        if cv2.waitKey(1) & 0xFF == ord('1'):
            ct[0] = not ct[0]
        if cv2.waitKey(1) & 0xFF == ord('2'):
                ct[1] = not ct[1]
        if cv2.waitKey(1) & 0xFF == ord('3'):
                ct[2] = not ct[2]
        if cv2.waitKey(1) & 0xFF == ord('1'):
                break
    cap.release()
    cv2.destroyAllWindows()

detection_phase()

In [42]:

cv2.destroyAllWindows()